In [5]:
# Step 1: Import Libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


In [6]:
# Step 2: Load the dataset
df = pd.read_csv('online_retail_II.csv', encoding='unicode_escape')
df.head()


c:\Users\kalya\AppData\Local\Programs\Python\Python312\Lib\encodings\unicode_escape.py:26: DeprecationWarning: invalid escape sequence '\O'
  return codecs.unicode_escape_decode(input, errors, final)
c:\Users\kalya\AppData\Local\Programs\Python\Python312\Lib\encodings\unicode_escape.py:26: DeprecationWarning: invalid escape sequence '\O'
  return codecs.unicode_escape_decode(input, errors, final)
c:\Users\kalya\AppData\Local\Programs\Python\Python312\Lib\encodings\unicode_escape.py:26: DeprecationWarning: invalid escape sequence '\O'
  return codecs.unicode_escape_decode(input, errors, final)
c:\Users\kalya\AppData\Local\Programs\Python\Python312\Lib\encodings\unicode_escape.py:26: DeprecationWarning: invalid escape sequence '\O'
  return codecs.unicode_escape_decode(input, errors, final)
c:\Users\kalya\AppData\Local\Programs\Python\Python312\Lib\encodings\unicode_escape.py:26: DeprecationWarning: invalid escape sequence '\O'
  return codecs.unicode_escape_decode(input, errors, final)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
# Step 3: Data Preprocessing
df.dropna(subset=['Invoice', 'Description', 'Quantity', 'Country'], inplace=True)
df = df[df['Quantity'] > 0]
df['Invoice'] = df['Invoice'].astype(str)


In [8]:
# Step 4: Filter for a single country (UK) and create a basket format
basket = (df[df['Country'] == "United Kingdom"]
          .groupby(['Invoice', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))


In [9]:
# Step 5: Encode data into 1 (purchased) and 0 (not purchased)
basket_sets = basket.applymap(lambda x: 1 if x >= 1 else 0)


C:\Users\kalya\AppData\Local\Temp\ipykernel_3880\3043879797.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket.applymap(lambda x: 1 if x >= 1 else 0)


In [14]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True, low_memory=True)


c:\Users\kalya\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [15]:
# Step 7: Generate Association Rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules.sort_values('lift', ascending=False)

# Display top 10 rules
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)


,antecedents,consequents,support,confidence,lift
663,(POPPY'S PLAYHOUSE BEDROOM ),(POPPY'S PLAYHOUSE LIVINGROOM ),0.010936,0.652597,48.462279
662,(POPPY'S PLAYHOUSE LIVINGROOM ),(POPPY'S PLAYHOUSE BEDROOM ),0.010936,0.812121,48.462279
665,(POPPY'S PLAYHOUSE KITCHEN),(POPPY'S PLAYHOUSE LIVINGROOM ),0.011290,0.634557,47.122556
664,(POPPY'S PLAYHOUSE LIVINGROOM ),(POPPY'S PLAYHOUSE KITCHEN),0.011290,0.838384,47.122556
127,(COFFEE MUG DOG + BALL DESIGN),(COFFEE MUG CAT + BIRD DESIGN),0.010147,0.743028,46.058958
126,(COFFEE MUG CAT + BIRD DESIGN),(COFFEE MUG DOG + BALL DESIGN),0.010147,0.629005,46.058958
660,(POPPY'S PLAYHOUSE BEDROOM ),(POPPY'S PLAYHOUSE KITCHEN),0.013194,0.787338,44.253433
661,(POPPY'S PLAYHOUSE KITCHEN),(POPPY'S PLAYHOUSE BEDROOM ),0.013194,0.741590,44.253433
765,(WOODEN STAR CHRISTMAS SCANDINAVIAN),(WOODEN TREE CHRISTMAS SCANDINAVIAN),0.011045,0.567039,41.031084
764,(WOODEN TREE CHRISTMAS SCANDINAVIAN),(WOODEN STAR CHRISTMAS SCANDINAVIAN),0.011045,0.799213,41.031084
